<a href="https://colab.research.google.com/github/Mogaveera/atmanomalydetection/blob/master/atm_anomaly_4_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
import tensorflow as tf
from keras.preprocessing import image
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Flatten, ZeroPadding2D,Reshape
from keras.layers.convolutional import (Conv2D, MaxPooling2D, AveragePooling2D, Conv3D, MaxPooling3D)
from keras.layers import Bidirectional, TimeDistributed
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix
from keras.utils import np_utils

print(tf.__version__)

SEQ_LEN = 30
BATCH_SIZE = 16
EPOCHS = 100

train_video_index = []
test_video_index = []

1.15.0


Using TensorFlow backend.


In [0]:
!git clone https://github.com/Mogaveera/atmanomaly.git

Cloning into 'atmanomaly'...
remote: Enumerating objects: 159, done.
remote: Total 159 (delta 0), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (159/159), 203.93 MiB | 48.39 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Checking out files: 100% (151/151), done.


In [0]:
def main():
    # Getting the data
    df = get_data('atmanomaly/Data/data_file.csv')
    
    # Clean the data
    #df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df)
    
    # Split the dataset into train and test
    train, test = split_train_test(df)
    
    # Encoding the dataset
    train_video_index = make_dataset(train, label_index, "train")
    test_video_index = make_dataset(test, label_index,"test")
    return (train_video_index, test_video_index)

In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
#def clean_data(df):
#    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
#    df = df[mask]`
#    return df

def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def video_to_frame(row, label_index, phase, not_created):
    input_f = []
    output_y = []
    index = 0
    cap = cv2.VideoCapture(os.path.join("atmanomaly/Data","anomaly_dataset",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".mp4")) 
    #print(str(row["class"].iloc[0]))
    #print(str(row["video_name"].iloc[0]))
    frameno = 1
    imgs = []
    length = 0
    seq = 12
    while (cap.isOpened()):
      ret, frame = cap.read()
      if ret:
        if length < seq:
          if frameno % 10 == 0:
            frameno = frameno + 1
            frame = preprocess_image(frame)
            frame = image.img_to_array(frame)
            frame = frame / 255
            imgs.append(frame)
            length = length + 1
          else:
            frameno = frameno + 1
        else:
          seq = seq + 12
          imgs1 = np.array(imgs)
          input_f.append(imgs1)
          output_y.append(label_index)
          del imgs[:]
      else:
        break

    if not_created:
      f = h5py.File(phase+'_4'+'.h5', 'w')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        f.create_dataset(phase, data=input_f1, maxshape=(None, 12, 299, 299, 3))
        f.create_dataset(phase+"_labels", data=output_y1, maxshape=(None, 4))
        f.close()
    else:
      hf = h5py.File(phase+'_4'+'.h5', 'a')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        hf[phase].resize((hf[phase].shape[0] + input_f1.shape[0]), axis = 0)
        hf[phase][-input_f1.shape[0]:] = input_f1

        hf[phase+"_labels"].resize((hf[phase+"_labels"].shape[0] + output_y1.shape[0]), axis = 0)
        hf[phase+"_labels"][-output_y1.shape[0]:] = output_y1
        hf.close()

    del input_f[:]
    del output_y[:]
    del imgs[:]
    cap.release()
    return index
    
    


def make_dataset(data, label_index, phase):
    video_index = [0]
    required_classes = ["Arson", "Burglary", "Fighting", "normal"]
   
    not_created = True
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
            index = required_classes.index(str(data.iloc[[i]]["class"].iloc[0]))
            label_index = np.zeros((4))
            label_index[index] = 1
            index = video_to_frame(data.iloc[[i]], label_index, phase, not_created)
            real_index = video_index[-1] + index
            video_index.append(real_index)
            if real_index > 0:
              not_created = False

    return video_index

In [0]:
train_video_index, test_video_index = main()

100%|██████████| 24/24 [00:56<00:00,  2.60s/it]


In [0]:
x_train = HDF5Matrix('train_4.h5', 'train')
y_train = HDF5Matrix('train_4.h5', 'train_labels')
x_test = HDF5Matrix('test_4.h5', 'test')
y_test = HDF5Matrix('test_4.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[210])
print(x_test.shape)
print(y_test.shape)
print(y_test[3])
print(train_video_index)
print(test_video_index)

(702, 12, 299, 299, 3)
(702, 4)
[0. 1. 0. 0.]
(176, 12, 299, 299, 3)
(176, 4)
[1. 0. 0. 0.]
[0, 3, 6, 10, 13, 16, 20, 21, 23, 26, 29, 32, 37, 42, 46, 49, 57, 61, 64, 68, 68, 73, 78, 83, 86, 90, 94, 96, 99, 101, 104, 105, 106, 107, 112, 114, 118, 123, 124, 128, 131, 132, 134, 136, 145, 150, 156, 160, 164, 167, 171, 174, 180, 187, 192, 198, 217, 221, 244, 266, 269, 271, 278, 280, 282, 285, 286, 289, 295, 296, 298, 301, 312, 316, 327, 333, 334, 345, 363, 369, 378, 381, 387, 390, 398, 412, 417, 424, 425, 429, 434, 440, 445, 448, 457, 461, 470, 477, 485, 492, 499, 504, 514, 521, 529, 545, 550, 558, 565, 569, 578, 585, 592, 600, 610, 628, 641, 648, 656, 666, 672, 681, 685, 693, 696, 702]
[0, 1, 3, 4, 5, 9, 12, 17, 21, 42, 48, 60, 61, 68, 75, 87, 92, 110, 116, 134, 140, 150, 157, 163, 176]


In [0]:
def model():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (12, 299, 299, 3)
    # Model.
    model = Sequential()
    
    model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1), input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1'))
      
    model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool2'))
    
    model.add(Conv3D(256, kernel_size=(3, 3, 3),activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1)))
    model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu', border_mode='valid', name='conv3b', subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool3'))
    
    model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', border_mode='valid', name='conv4a', subsample=(1, 1, 1)))
    model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', border_mode='valid', name='conv4b', subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 2, 2), border_mode='valid', name='pool4'))
    
    model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', border_mode='valid', name='conv5a', subsample=(1, 1, 1)))
    model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', border_mode='valid', name='conv5b', subsample=(1, 1, 1)))
    #model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(1, 3, 3), strides=(1, 3, 3), border_mode='valid', name='pool5'))
    
    # model.add(Dense(2048, activation='relu', name='fc7'))
    
    # model.add(Flatten())
    
    model.add(Reshape((12, 3 * 3 * 512), input_shape=(12, 3, 3, 512)))
    
    model.add(Bidirectional(LSTM(512)))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu', name='fc9'))
    model.add(Dropout(0.5))
    
    model.add(Dense(4, activation='softmax'))
    #model.add(Dense(10, activation='softmax'))"""
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    callback_list = [checkpoint]
    
    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    #return model, callback_list
    #model.compile(optimizer = tf.train.AdamOptimizer(),
    #          loss = 'categorical_crossentropy',
    #        metrics=['accuracy'])
    return model, callback_list


In [0]:
model, callback_list = model()
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, kernel_size=(3, 3, 3), activation="relu", name="conv1", input_shape=(12, 299, ..., strides=(1, 1, 1), padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, kernel_size=(3, 3, 3), activation="relu", name="conv2", strides=(1, 1, 1), padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool2", padding="valid


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 12, 299, 299, 64)  5248      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 12, 149, 149, 64)  0         
_________________________________________________________________
conv2 (Conv3D)               (None, 12, 149, 149, 128) 221312    
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 12, 74, 74, 128)   0         
_________________________________________________________________
conv3a (Conv3D)              (None, 12, 74, 74, 256)   884992    
_________________________________________________________________
conv3b (Conv3D)              (None, 12, 72, 72, 256)   590080    
__

In [0]:
model.fit(x_train, y_train, batch_size = 6, epochs = 20,verbose = 2, validation_data = (x_test, y_test), shuffle = 'batch',callbacks=callback_list)
model.save("atm_anomaly_4classes_usingconv3d.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 702 samples, validate on 176 samples
Epoch 1/20





 - 258s - loss: 1.3820 - acc: 0.3191 - val_loss: 1.3538 - val_acc: 0.3409
Epoch 2/20
 - 297s - loss: 1.3437 - acc: 0.3490 - val_loss: 1.2532 - val_acc: 0.3409
Epoch 3/20
 - 207s - loss: 1.2666 - acc: 0.4017 - val_loss: 1.1314 - val_acc: 0.4886
Epoch 4/20
 - 207s - loss: 1.1806 - acc: 0.4786 - val_loss: 1.0097 - val_acc: 0.5568
Epoch 5/20
 - 207s - loss: 1.0177 - acc: 0.5527 - val_loss: 0.9502 - val_acc: 0.6875
Epoch 6/20
 - 208s - loss: 0.8927 - acc: 0.6011 - val_loss: 0.9856 - val_acc: 0.4886
Epoch 7/20
 - 207s - loss: 0.8509 - acc: 0.6125 - val_loss: 0.9367 - val_acc: 0.5966
Epoch 8/20
 - 207s - loss: 0.7087 - acc: 0.6966 - val_loss: 1.0774 - val_acc: 0.6307
Epoch 9/20
 - 207s - loss: 0.6990 - acc: 0.7123 - val_loss: 1.0409 - val_acc: 0.6477
Epoch 10/20
 - 207s - loss: 0.5190 - acc: 0.8020 - val_loss: 1.1275 - val_acc: 0.6080
E

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=6)
print("accuracy: ", test_acc)

176/176 [==============================] - 20s 113ms/step
accuracy:  0.6647727269340645
accuracy:  0.6647727269340645


In [0]:
img = []
img.append(x_test[5])
imgs1 = np.array(img)

pred = model.predict_classes(imgs1)
print(pred)

[2]
[2]
